In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64
#create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader  = DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
    print("Shape of X [N, C, H, W]:", X.shape)
    print("Shape of y: ", y.shape, y.dtype )
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [11]:
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch %100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} \t\t [{current:>5d}/{size:>5d}]")

In [20]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Abgloss:{test_loss:>8f} \n')


In [21]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------------------
loss: 2.057801 		 [    0/60000]
loss: 2.051636 		 [ 6400/60000]
loss: 1.961214 		 [12800/60000]
loss: 2.031689 		 [19200/60000]
loss: 1.904892 		 [25600/60000]
loss: 1.848259 		 [32000/60000]
loss: 1.965978 		 [38400/60000]
loss: 1.812493 		 [44800/60000]
loss: 1.869190 		 [51200/60000]
loss: 1.908993 		 [57600/60000]
Test Error: 
 Accuracy: 51.7%, Abgloss:1.815898 

Epoch 2
----------------------------------------
loss: 1.854156 		 [    0/60000]
loss: 1.835012 		 [ 6400/60000]
loss: 1.696527 		 [12800/60000]
loss: 1.826766 		 [19200/60000]
loss: 1.660179 		 [25600/60000]
loss: 1.600594 		 [32000/60000]
loss: 1.789973 		 [38400/60000]
loss: 1.586173 		 [44800/60000]
loss: 1.678487 		 [51200/60000]
loss: 1.778748 		 [57600/60000]
Test Error: 
 Accuracy: 57.5%, Abgloss:1.630491 

Epoch 3
----------------------------------------
loss: 1.688710 		 [    0/60000]
loss: 1.663669 		 [ 6400/60000]
loss: 1.500454 		 [12800/60000]
loss: 1.681647 		

In [22]:
torch.save(model.state_dict(), "model.pth")
print("Save PyTorch Model State to model.pth")

Save PyTorch Model State to model.pth


In [24]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [39]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]
model.eval()
x,y=test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    print(pred.argmax(1).item(),y)
    predicted, actual = classes[pred.argmax(1).item()], classes[y]
    print(f"Predected: '{predicted}' Actual: '{actual}'")

9 9
Predected: 'Ankle boot' Actual: 'Ankle boot'
